### Install dependencies

In [430]:
#!pip install NetworkX
#!pip install biopython
#!pip install blosum

### Imports

In [431]:
from Bio.Blast.Applications import NcbiblastpCommandline
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import uuid
from Bio.Seq import Seq
import blosum as bl
import re
import pandas as pd
import numpy as np
import tqdm
import torch
import pandas as pd
from Bio import SeqIO
import networkx as nx

### Extract data

In [432]:
# import tarfile
  
# # open file
# file = tarfile.open('temp_outs.tar.gz')
  
# # extracting file
# file.extractall('./')
  
# file.close()

### Align sequences with ESM (code mostly from Mahdi)

In [433]:
seq_1 = torch.load(f"temp_outs\outputs\QBZ81690.1.pt")
seq_2 = torch.load(f"temp_outs\outputs\AGO83968.2.pt")
seq_1

{'label': 'QBZ81690.1',
 'representations': {36: tensor([[ 0.0305, -0.2712, -0.0361,  ..., -0.1840, -0.0522, -0.0922],
          [ 0.1543, -0.1236, -0.0234,  ..., -0.0630, -0.2551,  0.0359],
          [ 0.0661, -0.0476, -0.0673,  ..., -0.0502, -0.2257,  0.0975],
          ...,
          [ 0.0282,  0.0571,  0.0858,  ..., -0.0320, -0.1567,  0.0178],
          [ 0.0330, -0.1428,  0.0135,  ..., -0.0338, -0.0357, -0.1182],
          [-0.1102, -0.0696, -0.1198,  ..., -0.0898, -0.1002, -0.0122]])},
 'mean_representations': {36: tensor([ 0.0314, -0.0491, -0.0131,  ..., -0.0286, -0.0542,  0.0002])}}

In [434]:
# Compute all pairwise distances
data = []
for i in range(seq_1["representations"][36].shape[0]):
    line = []
    for j in range(seq_2["representations"][36].shape[0]):
        v1 = seq_1["representations"][36][i]
        v2 = seq_2["representations"][36][j]
        line.append(round(torch.cdist(v1.unsqueeze(0), v2.unsqueeze(0)).item(), 2))
    data.append(line)
    

# Write the matrix to the excel sheet    
data = pd.DataFrame(data)
data

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,4.85,7.74,7.44,7.62,7.52,7.21,7.61,7.44,7.18,7.28,...,6.24,6.36,6.65,6.82,6.46,6.24,6.15,6.01,6.34,6.48
1,7.78,7.29,7.96,8.19,7.94,7.76,8.02,8.16,6.97,7.92,...,7.32,7.48,6.56,7.47,7.37,7.08,6.90,6.80,7.03,7.06
2,7.28,7.33,7.06,7.43,6.92,6.78,7.45,7.38,7.08,6.97,...,6.10,6.50,6.66,6.32,6.13,6.07,5.90,5.94,6.17,6.29
3,7.18,7.67,7.11,7.23,7.23,6.96,7.19,7.27,7.15,7.04,...,5.16,5.61,6.76,6.65,4.90,4.80,5.86,5.65,6.08,6.43
4,6.94,7.04,6.88,7.00,6.83,6.72,7.06,6.98,6.90,6.79,...,5.89,6.30,6.44,6.50,5.93,5.53,5.38,5.39,5.81,6.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,6.73,7.26,6.89,7.00,6.54,5.83,6.99,6.91,6.42,6.58,...,5.15,5.68,5.81,5.82,5.24,4.84,3.61,4.52,5.04,5.47
95,7.01,7.47,7.12,7.47,6.89,6.76,7.27,7.35,6.78,6.77,...,5.82,6.21,6.13,6.37,5.70,5.29,5.10,4.82,5.43,5.90
96,6.74,7.25,6.88,7.10,6.58,5.84,6.94,6.98,6.63,6.58,...,5.30,5.81,5.82,5.88,5.28,4.91,3.84,4.44,4.80,5.37
97,7.12,7.52,7.38,7.54,7.10,6.96,7.25,7.33,6.98,7.20,...,5.98,6.23,6.38,6.48,5.85,5.49,5.29,4.96,5.08,5.76


In [435]:
seqs = SeqIO.to_dict(SeqIO.parse("test/ab.fa", 'fasta'))

seq_ids = list(seqs.keys())
seq_ids = [seq_ids[0]] + [seq_ids[1]]
seq_ids

['QBZ81690.1', 'AGO83968.2']

In [436]:
top_n = 5
rows = pd.DataFrame({n: data.T[col].nsmallest(top_n).index.tolist() 
                  for n, col in enumerate(data.T)}).T

cols = pd.DataFrame({n: data[row].nsmallest(top_n).index.tolist() 
                  for n, row in enumerate(data)})

matches = set()
for i, n_cols in enumerate(rows.values):
    for c in n_cols:
        if i in cols.iloc[:, c].values:
            matches.add((i, int(c)))

temp_set = set()
for match in matches:
    if match[0] > 0 and match[1] > 0 and (seqs[seq_ids[0]][match[0]-1] ==  seqs[seq_ids[1]][match[1]-1]):
            temp_set.add((match[0]-1, match[1]-1))
            #if sheet[(match[0]+1, match[1]+1)].color is None:
            #    sheet[(match[0]+1, match[1]+1)].color = "#82C341"
    if match[0] < len(seqs[seq_ids[0]]) - 1  and match[1] < len(seqs[seq_ids[1]])-1 and  (seqs[seq_ids[0]][match[0]+1] ==  seqs[seq_ids[1]][match[1]+1]):
            temp_set.add((match[0]+1, match[1]+1))
            #if sheet[(match[0]+1, match[1]+1)].color is None:
            #    sheet[( match[0]+3, match[1]+3)].color = "#82C341"

matches = matches.union(temp_set)
list(matches)[0:10]

[(38, 23),
 (16, 84),
 (78, 68),
 (96, 104),
 (36, 71),
 (27, 13),
 (18, 44),
 (79, 60),
 (29, 41),
 (74, 73)]

In [437]:
def build_paths_graph(matches):
    graph = nx.DiGraph()
    dag = {}
    max_depth = max([x[0] for x in matches])
    sorted_matches = sorted(matches, key=lambda x: x[1])
    for i in range(len(sorted_matches) - 1):
        last_depth =  max_depth
        dag[sorted_matches[i]] = []
        #print(f"Comparing {sorted_matches[i]} and {sorted_matches[j]}")
        for j in range(i+1, len(sorted_matches)):
            if (sorted_matches[j][0]) < last_depth and  (sorted_matches[j][0] >  sorted_matches[i][0]):
                dag[sorted_matches[i]].append(sorted_matches[j])
                seq_1_idx, seq_2_idx = sorted_matches[j]
                graph.add_edge(sorted_matches[i], sorted_matches[j], weigth=data.iloc[seq_1_idx, seq_2_idx])
                last_depth = sorted_matches[j][0]
    return graph


graph = build_paths_graph(matches)
esmAlignment = nx.dag_longest_path(graph)
#len(esmAlignment)
esmAlignment[0:10]

[(0, 0),
 (9, 13),
 (10, 14),
 (12, 15),
 (13, 22),
 (14, 25),
 (26, 28),
 (29, 41),
 (32, 47),
 (37, 49)]

### Align sequences with BLAST

In [438]:
seqs = SeqIO.to_dict(SeqIO.parse("test/ab.fa", 'fasta'))
str(seqs['QBZ81690.1'].seq)

'MGQANTTIKGYSAVSQDALFATYGITEGDYQAKASAAIERIRAMPEGYASPEDRAAAINAIRTGTCGDDTELLTRVRAALDRWQRDCASTGHALKSKVV'

In [439]:
data = ['', 'QBZ81690.1', '', 'AGO83968.2']

seq1 = SeqRecord(seqs[data[1]].seq,
                 id=seqs[data[1]].id)
seq2 = SeqRecord(seqs[data[3]].seq,
                 id=seqs[data[3]].id)
SeqIO.write(seq1, f"seq_1.fasta", "fasta")
SeqIO.write(seq2, f"seq_2.fasta", "fasta")

str(seq1.seq)

'MGQANTTIKGYSAVSQDALFATYGITEGDYQAKASAAIERIRAMPEGYASPEDRAAAINAIRTGTCGDDTELLTRVRAALDRWQRDCASTGHALKSKVV'

In [440]:
output = NcbiblastpCommandline(query=f"seq_1.fasta", subject=f"seq_2.fasta", outfmt="6 qstart qseq sstart sseq")()[0]
if len(output) < 2:
    output = f"{seqs[data[1]].id}\t{seqs[data[3]].id}\t NO_HIT"
#output = output.split("\n")
print(output)

29	DYQAKASAAIERIRAMPEGYASPEDRAAAINAIRTGTCGDDTELLTRVRAALDRWQRDCAS	41	EYAELKEKTIRIIVALPTQYLSADKGRAAIEAIRSGEPGREA-LYDEVWDAAMAYSRDIAA
16	QDALF	80	REALY



In [441]:
# Enumerate the BLAST output string based on the 
def GetBlastIndices(start, blast):
    start = int(start) - 1 # BLAST outputs 1-based indices
    
    indices = [i for i in range(start, start + len(blast))]
    
    gaps = [i for i, c in enumerate(blast) if c == "-"]
    
    for gap in gaps:
        indices.insert(gap, -1)
        
    return indices[0:len(blast)]

GetBlastIndices(2, "abcd-efg")

[1, 2, 3, 4, -1, 5, 6, 7]

In [442]:
def GetBlastAlignRegion(start1, blast1, start2, blast2):
    i1 = GetBlastIndices(start1, blast1)
    i2 = GetBlastIndices(start2, blast2)
    
    return list(zip(i1, i2))

testBlastIndices = GetBlastAlignRegion(2, "abcde", 1, "wxy-z")
testBlastIndices

[(1, 0), (2, 1), (3, 2), (4, -1), (5, 3)]

In [443]:
def BlastAlign(file1, file2):
    blastRaw = NcbiblastpCommandline(query=file1, subject=file2, outfmt="6 qstart qseq sstart sseq")()[0]
    blastLines = blastRaw.splitlines()
    blastFields = [re.split(r'\t+', line) for line in blastLines]
    
    alignments = [GetBlastAlignRegion(*blast) for blast in blastFields]
    
    return alignments
    
blastAlignments = BlastAlign(f"seq_1.fasta", f"seq_2.fasta")
#blastAlignments
[seq[0:10] for seq in blastAlignments]

[[(28, 40),
  (29, 41),
  (30, 42),
  (31, 43),
  (32, 44),
  (33, 45),
  (34, 46),
  (35, 47),
  (36, 48),
  (37, 49)],
 [(15, 79), (16, 80), (17, 81), (18, 82), (19, 83)]]

### Concatenate BLAST sequences if they don't overlap

In [444]:
def BlastConcat(aligns):
    alignUnzip = [list(zip(*align)) for align in aligns]
    
    query = list(alignUnzip[0][0])
    subject = list(alignUnzip[0][1])
    
    alignWasUsed = [True]
    
    for q,s in alignUnzip[1:]:
        if (max(min(query), min(q)) > min(max(query), max(q))) and (max(min(subject), min(s)) > min(max(subject), max(s))):
        #if set(query).isdisjoint(set(q)) and set(subject).isdisjoint(set(s)):
            query = query + list(q)
            subject = subject + list(s)
            alignWasUsed.append(True)
        else:
            alignWasUsed.append(False)
            
    alignsUsed = [i for i,b in enumerate(alignWasUsed) if b]
            
    return list(zip(query, subject)), alignsUsed
    
#BlastConcat([[(1,1), (1,2), (2,3), (5,5)], [(1,1), (2,2)], [(4,4), (6,7)]])
blastBestAlign, blastAlignsUsed = BlastConcat(blastAlignments)
blastBestAlign[0:10], blastAlignsUsed

([(28, 40),
  (29, 41),
  (30, 42),
  (31, 43),
  (32, 44),
  (33, 45),
  (34, 46),
  (35, 47),
  (36, 48),
  (37, 49)],
 [0])

### Combine the results

In [445]:
# If there are indices of -1, we want to fill them with the previous index + 0.5 to keep them sorted in place.
def CreateSortableIndex(indices):
    result = []
    j = -1
    for i in indices:
        if i < 0:
            result.append(j + 0.5)
        else:
            result.append(float(i))
            j = i
    return result

CreateSortableIndex([0, 1, 2, -1, 3, -1])

[0.0, 1.0, 2.0, 2.5, 3.0, 3.5]

In [446]:
def InterweaveAlignments(align1, align2):
    (indices11, indices12) = map(list, zip(*align1))
    (indices21, indices22) = map(list, zip(*align2))
    
    sort11 = CreateSortableIndex(indices11)
    sort12 = CreateSortableIndex(indices12)
    sort21 = CreateSortableIndex(indices21)
    sort22 = CreateSortableIndex(indices22)
    
    df1 = pd.DataFrame(data={"Sort": zip(sort11, sort12), "QLoc": indices11, "SLoc": indices12, "Align1": True})
    df2 = pd.DataFrame(data={"Sort": zip(sort21, sort22), "QLoc": indices21, "SLoc": indices22, "Align2": True})
    
    df = pd.merge(df1, df2, on=["Sort", "QLoc", "SLoc"], how="outer", sort=True).fillna(False)
    df = df.drop(columns=["Sort"])
    
    return df
    
testAligns = InterweaveAlignments(blastBestAlign, esmAlignment)
testAligns[0:10]

,QLoc,SLoc,Align1,Align2
0,0,0,False,True
1,9,13,False,True
2,10,14,False,True
3,12,15,False,True
4,13,22,False,True
5,14,25,False,True
6,26,28,False,True
7,28,40,True,False
8,29,41,True,True
9,30,42,True,False


In [447]:
#Identify the positive matches by looking directly in the score matrix? https://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt
blosumDict = dict(bl.BLOSUM(62))

def Blosum(a, b):
    key = str(a.upper()) + str(b.upper())
    return blosumDict.get(key, np.nan)

Blosum("A", "N")

-2.0

In [448]:
def CompareAlignments(query, subject, align1, align2):
    df = InterweaveAlignments(align1, align2)
    df["Query"] = [query[i] if (i >= 0) else "-" for i in df.QLoc]
    df["Subject"] = [subject[i] if (i >= 0) else "-" for i in df.SLoc]
    df["Score"] = [Blosum(a,b) for (a,b) in list(zip(df.Query, df.Subject))]
    
    return df

print(len(seq1.seq))
print(len(seq2.seq))
print(max(blastBestAlign))
print(max(esmAlignment))
bestAlignDF = CompareAlignments(seq1.seq, seq2.seq, blastBestAlign, esmAlignment)
bestAlignDF[50:60]

99
108
(88, 99)
(96, 105)


,QLoc,SLoc,Align1,Align2,Query,Subject,Score
50,66,78,True,True,G,G,6.0
51,67,79,True,False,D,R,-2.0
52,68,80,True,False,D,E,2.0
53,69,81,True,False,T,A,0.0
54,70,-1,True,False,E,-,NaN
55,71,82,True,False,L,L,4.0
56,72,82,False,True,L,L,4.0
57,72,83,True,False,L,Y,-1.0
58,73,84,True,False,T,D,-1.0
59,74,85,True,False,R,E,0.0


In [449]:
alignmentDFs = [CompareAlignments(seq1.seq, seq2.seq, blastAlign, esmAlignment) for blastAlign in blastAlignments]
alignmentDFs[0]

,QLoc,SLoc,Align1,Align2,Query,Subject,Score
0,0,0,False,True,M,M,5.0
1,9,13,False,True,G,G,6.0
2,10,14,False,True,Y,Y,7.0
3,12,15,False,True,A,A,4.0
4,13,22,False,True,V,V,4.0
...,...,...,...,...,...,...,...
75,88,99,True,False,S,A,1.0
76,90,100,False,True,G,G,6.0
77,92,102,False,True,A,A,4.0
78,94,104,False,True,K,K,5.0


### Render the comparison

In [450]:
def GetGaps(df):
    gaps = []
    q = 0
    s = 0
    for i,row in df.iterrows():
        if row.QLoc > (q + 1) and row.SLoc > (s + 1):
            gaps.append(i)
        q = row.QLoc
        s = row.SLoc
    
    return gaps

GetGaps(bestAlignDF)

[1, 6, 7, 77, 78]

In [451]:
# Make a string corresponding to a single line for our visual
def SeqString(conditions, values, gaps, blankChar=" ", gapStr=" "):
    chars = np.select([conditions, ~conditions], [values, blankChar])
    lineStr = "".join(chars)
    for i in reversed(gaps):
        lineStr = lineStr[:i] + gapStr + lineStr[i:]
    return lineStr

SeqString(pd.Series([False, True, True, False, True]), [1, 2, 3, 4, 5], [2])

' 2 3 5'

In [452]:
def RenderAlignments(df):
    positive = pd.DataFrame(np.where(df.Score > 0.0, "+", None))
    perfect = pd.DataFrame(np.where(df.Query == df.Subject, "*", None))
    scorestring = perfect.combine_first(positive)[0].fillna(" ")
    
    gaps = GetGaps(bestAlignDF)
    
    a1_condition = [df.Align1, ~df.Align1]
    a2_condition = [df.Align2, ~df.Align2]
    
    #alignment 1
    print(SeqString(df.Align1, df.Query, gaps))
    print(SeqString(df.Align1, scorestring, gaps))
    print(SeqString(df.Align1, df.Subject, gaps))
    
    #anchors
    print(SeqString(df.Align1&df.Align2, "|", gaps))
    
    #alignment 2
    print(SeqString(df.Align2, df.Query, gaps))
    print(SeqString(df.Align2, scorestring, gaps))
    print(SeqString(df.Align2, df.Subject, gaps))

### Results

In [453]:
# Show comparison against top BLAST result, with any other compatible BLAST HSPs concatenated on
print("Best non-overlapping BLAST HSPs concatenated (top) vs ESM-based alignment (bottom)\n\n")
RenderAlignments(bestAlignDF)

Best non-overlapping BLAST HSPs concatenated (top) vs ESM-based alignment (bottom)


          DYQAK ASAAIE R IRAMPEGYASPE DRA AAINAIRTGTCGDDTEL LTRVRAA L DRWQRDCAS      
          +*        *    * *+*  * * +     *** ***+*  * +  *    *  *     + ** *+      
          EYAEL KEKTIR I IVALPTQYLSAD KGR AAIEAIRSGEPGREA-L YDEVWDA A MAYSRDIAA      
           |        |      | | || |    |  ||||||||||||         |        ||||||       
M GYAVS E  Y   K    I R I  A P GY S  D R AAAINAIRTGTCG     L   V   L D  WQRDCA G A KK
* ***** *  *   *    * * *  * *  * *  *   **** ***+*  *     *   *     *  + ** * * * * 
M GYAVS E  Y   K    I R I  A P QY S  D G AAAIEAIRSGEPG     L   V   W D  YSRDIA G A KP


In [460]:
print("If any BLAST alignments were unable to concatenate to the above result, they will be displayed here:\n")

# Show comparison any of top 10 BLAST HSPs that *weren't* used in the concatenation above
for i, alignments in enumerate(alignmentDFs):
    if not i in blastAlignsUsed:
        print(f"\n\nBLAST HSP #{i+1} (top) vs ESM-based alignment (bottom)\n\n")
        RenderAlignments(alignments)
        print("\n")

If any BLAST alignments were unable to concatenate to the above result, they will be displayed here:



BLAST HSP #2 (top) vs ESM-based alignment (bottom)


        Q DALF                                          
        + +**+                                          
        R EALY                                          
                                                        
M GYAVS       EYKIRIAPGYSDRAAAINAIRTGTCGLVLDWQRDCAGAKK  
* *****       ******** *** **** ***+*  *** *+ ** ****   
M GYAVS       EYKIRIAPQYSDGAAAIEAIRSGEPGLVWDYSRDIAGAKP  




### To Do / Questions: 
1. How do we want to show large gaps that occur in both alignments? Currently just a single space.
2. Do we want to truncate the range to only that covered by the BLAST HSP?
2. Do we want to show indices in the visual? In what way?
3. Could concatenate BLAST HSPs together in a more intelligent way. Currently uses a greedy algorithm to combine the top-scoring non-overlapping BLAST HSPs. Could instead find the optimal combination. 
4. Can BLAST HSPs have a reversal of Query and/or Subject?